### VectorDB -> 벡터라이징 데이터 활용 -> LAGChain 구성

In [3]:
# 가장 간단한 설정
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

import numpy as np
np.set_printoptions(threshold=np.inf)

# RAG 프로젝트용 권장 설정
import pandas as pd
import numpy as np

# 데이터프레임 출력 제한 해제
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 벡터 출력 제한 해제
np.set_printoptions(threshold=np.inf)

print("RAG 프로젝트 출력 설정 완료!")

# 특정 출력에서만 전체 표시
# with pd.option_context('display.max_rows', None):
#     print(your_dataframe)

import pandas as pd

# 임시로 설정 변경
old_max_rows = pd.get_option('display.max_rows')
old_max_columns = pd.get_option('display.max_columns')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#print(your_dataframe)

# 원래 설정으로 복원
pd.set_option('display.max_rows', old_max_rows)
pd.set_option('display.max_columns', old_max_columns)

RAG 프로젝트 출력 설정 완료!


In [4]:
# python-dotenv 패키지: 환경 변수를 .env 파일에서 로드하는 라이브러리
# - 보안이 필요한 API 키 등을 관리하는데 사용
# - pip install python-dotenv로 설치 가능
from dotenv import load_dotenv

# load_dotenv(): .env 파일의 환경 변수를 현재 실행 환경에 로드하는 함수
# - 반환값: 성공 시 True, 실패 시 False
load_dotenv(override=True)

True

### Prompt - 버전 1.0

In [123]:
from langchain.prompts import PromptTemplate

template = """
당신은 클라우드 사용 매뉴얼을 기반으로 질문에 답변하는 전문가입니다.

다음은 검색된 문서 내용입니다:
====================
{context}
====================

위 내용을 참고하여 다음 질문에 정확하고 간결하게 답변하세요:
질문: {question}

- 검색된 문서의 내용을 벗어나지 마세요.

"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

### Prompt - 버전 1.5 (출력값을 정렬 및 사람이 보고 편한 구조로 개선)

In [5]:
from langchain.prompts import PromptTemplate

template = """
당신은 클라우드 사용 매뉴얼을 기반으로 질문에 답변하는 전문가입니다.

다음은 검색된 문서 내용입니다:
====================
{context}
====================

질문: {question}

위 내용을 참고하여 다음 조건을 지켜 답변하세요:

1. 반드시 마크다운 형식으로 정리하세요.
2. 단계별 설명이 필요하면 **숫자 목록**으로 나누어 설명하세요.
3. UI 경로는 `"Menu > Submenu"` 형식으로 표현하세요.
4. 전체 답변은 500자 이내로 간결하게 작성하세요.
5. 검색된 문서의 내용을 벗어나지 마세요.

답변:

"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

In [9]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
import os


# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# ChromaDB 벡터스토어 연결
# persist_directory는 ChromaDB가 저장된 디렉토리 경로
persist_directory = "./chroma_db"

# 기존 컬렉션에서 벡터스토어 로드
vectorstore = Chroma(
    collection_name="manual_user_collection",
    embedding_function=embeddings,
    persist_directory=persist_directory
)

print(f"벡터스토어가 성공적으로 로드되었습니다.")
print(f"컬렉션 이름: {vectorstore._collection.name}")
print(f"컬렉션 내 문서 수: {vectorstore._collection.count()}")

/var/folders/3k/glrtc_n92r1b88148508h8rc0000gn/T/ipykernel_86314/4021498024.py:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
/var/folders/3k/glrtc_n92r1b88148508h8rc0000gn/T/ipykernel_86314/4021498024.py:14: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectorstore = Chroma(


벡터스토어가 성공적으로 로드되었습니다.
컬렉션 이름: manual_user_collection
컬렉션 내 문서 수: 97


In [10]:
import json
from typing import List
from langchain_core.documents import Document

def parse_image_urls(image_urls_raw):
    if isinstance(image_urls_raw, str):
        try:
            image_urls = json.loads(image_urls_raw)
        except json.JSONDecodeError:
            image_urls = []
    elif isinstance(image_urls_raw, list):
        image_urls = image_urls_raw
    else:
        image_urls = []
    return image_urls

def format_images(image_urls):
    return "\n".join([f"- ![image]({url})" for url in image_urls])

def format_result_with_metadata(question: str, answer: str, docs: List[Document]):
    md = f"### 💬 질문\n{question.strip()}\n\n"
    md += f"### 🧠 답변\n{answer.strip()}\n\n"
    md += "---\n\n"
    md += f"### 📎 관련 문서 및 이미지\n"

    for i, doc in enumerate(docs, 1):
        url = doc.metadata.get('source_url', '')
        image_urls_raw = doc.metadata.get('image_urls', '[]')

        md += f"\n**🔗 문서 출처 {i}**\n"
        md += f"[웹 메뉴얼 보기]({url})\n"

        image_urls = parse_image_urls(image_urls_raw)
        if image_urls:
            markdown_images = format_images(image_urls[:3])
            md += "이미지:\n"
            md += markdown_images + "\n"

    return md

In [13]:
from langchain_community.llms import OpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda


# LLM 초기화
llm = ChatOpenAI(
            model="gpt-4o-mini",  # 사용할 모델 이름을 지정 가능
            temperature=0,        # temperature는 0~1 사이의 값으로, 0에 가까울수록 일관된 답변을, 1에 가까울수록 다양하고 창의적인 답변을 생성합니다
            max_tokens=100,       # 생성할 최대 토큰 수
            )

# Retriever 생성
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [14]:
# 검색된 문서를 저장할 전역 변수 (List[Document] 타입)
from typing import List
from langchain.schema import Document

#retriever_docs_backup = []
retriever_docs_backup: List[Document] = []
# section 필드만 추출하는 함수
def extract_section(docs):
    global retriever_docs_backup
    retriever_docs_backup = docs
    return {"context": [doc.page_content for doc in docs]}


# LCEL 체인 구성
chain = (
    {"context": retriever | RunnableLambda(extract_section), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 체인 테스트
question = "로그인은 어떻게 해야해?"
response = chain.invoke(question)

# 결과 포맷팅(LLM 출력 + 벡터DB 메타데이터)
final_output = format_result_with_metadata(question, response, retriever_docs_backup)

from IPython.display import display, Markdown

# display(Markdown(final_output))
print("답변:", final_output)

with open(f"output_{question}.md", "w", encoding="utf-8") as f:
    f.write(final_output)



#print("질문:", question)
#print("답변:", response)

답변: ### 💬 질문
로그인은 어떻게 해야해?

### 🧠 답변
## 로그인 방법

로그인 절차는 다음과 같습니다:

1. **브라우저 접속**
   - 브라우저 URL 창에 `httpstkctl.tg-cloud.co.kr`을 입력하여 포탈에 접속합니다.

2. **로그인 인증**
   - 로그인 창에서 ID와 PW를 입력합니다.
     - **ID**: username 또는 email 형식으로 입력
     - **PW**: 로그인 계정의 비밀번호

3. **포

---

### 📎 관련 문서 및 이미지

**🔗 문서 출처 1**
[웹 메뉴얼 보기](https://doc.tg-cloud.co.kr/manual/console/login/login/login)
이미지:
- ![image](https://doc.tg-cloud.co.kr/manual/console/login/login/img/login_url.png)
- ![image](https://doc.tg-cloud.co.kr/manual/console/login/login/img/login_input.png)
- ![image](https://doc.tg-cloud.co.kr/manual/console/login/login/img/login_success.png)

**🔗 문서 출처 2**
[웹 메뉴얼 보기](https://doc.tg-cloud.co.kr/manual/console/firstUser/login/login)
이미지:
- ![image](https://doc.tg-cloud.co.kr/manual/console/firstUser/login/img/login_url.png)
- ![image](https://doc.tg-cloud.co.kr/manual/console/firstUser/login/img/login_input.png)
- ![image](https://doc.tg-cloud.co.kr/manual/console/firstUser/login/img/login_first_men

In [ ]:
import json
from typing import List
from langchain_core.documents import Document  # 꼭 필요!

def parse_image_urls(image_urls_raw):
    # 1. 문자열일 경우 → json.loads
    if isinstance(image_urls_raw, str):
        try:
            return json.loads(image_urls_raw)
        except json.JSONDecodeError:
            return []

    # 2. 이미 리스트인 경우 → 그대로 반환
    elif isinstance(image_urls_raw, list):
        return image_urls_raw

    # 3. 그 외 (NoneType, dict 등) → 빈 리스트
    return []


# 🧾 응답 + 메타데이터 마크다운 포맷
def format_result_with_metadata(question: str, answer: str, docs: List[Document]):
    md = f"### 💬 질문\n{question.strip()}\n\n"
    md += f"### 🧠 답변\n{answer.strip()}\n\n"
    md += "---\n\n"
    md += f"### 📎 관련 문서 및 이미지\n"

    for i, doc in enumerate(docs, 1):
        url = doc.metadata.get('source_url', '#')
        image_urls_raw = doc.metadata.get('image_urls', [])

        md += f"\n**🔗 문서 출처 {i}**\n"
        md += f"[웹 메뉴얼 보기]({url})\n"

        image_urls = parse_image_urls(image_urls_raw)
        if image_urls:
            md += "이미지:\n"
            for img in image_urls[:3]:  # 최대 3개 출력
                md += f"- ![image]({img})\n"

    return md

# 🧠 응답 생성 체인 함수 (예외처리 포함)
def generate_response(user_input):
    question = str(user_input).strip()

    if not question:
        return "❌ 질문이 비어 있습니다. 내용을 입력해주세요."

    try:
        # 🔁 실제 코드에서는 외부 정의된 객체 사용
        docs = retriever.invoke(question)
        context = extract_section(docs)
        answer = chain.invoke({
            "question": question,
            "context": context["context"]
        })
        return format_result_with_metadata(question, answer, docs)

    except Exception as e:
        return f"🚨 오류 발생: {type(e).__name__} - {str(e)}"




response_md = generate_response("TKSCTL로 터미널 접속하는 법 알려줘")
print(response_md)

import gradio as gr

# Gradio 인터페이스 구성
# with gr.Blocks(title="문서 기반 QA 봇") as demo:
#     gr.Markdown("## 📘 문서 기반 QA 챗봇\n질문을 입력하면 관련 문서를 기반으로 답변합니다.")
    
#     with gr.Row():
#         txt_input = gr.Textbox(lines=2, label="질문 입력")
#         submit_btn = gr.Button("질문하기")

#     output_area = gr.Markdown(label="📋 답변 결과")

#     submit_btn.click(
#         fn=generate_response,
#         inputs=txt_input,
#         outputs=output_area
#     )

# demo.launch()


🚨 오류 발생: TypeError - expected string or buffer


In [26]:
demo.close()

Closing server running on port: 7864
